<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\451024427.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432 

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_3_1 = f'''
SELECT 
    COUNT(*) AS cnt_vac
FROM 
    vacancies
'''

In [5]:
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3851344990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,cnt_vac
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
query_3_2 = f'''
SELECT 
    COUNT(*) AS cnt_employers 
FROM 
    employers
'''

In [7]:
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3588252056.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,cnt_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query_3_3 = f'''
SELECT 
    COUNT(*) AS cnt_areas 
FROM 
    areas
'''

In [9]:
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\2018224393.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,cnt_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
query_3_4 = f'''
SELECT 
    COUNT(*) AS cnt_indistr
FROM 
    industries
'''

In [11]:
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\2172479966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,cnt_indistr
0,294


***

ВЫВОДЫ: Согласно предварительному анализу, в базе данных почти 50 тыс. вакансий, около 23,5 тыс. работодателей с почти 300 видами деятельности из 1362 регионов.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query_4_1 = f'''
SELECT 
    a.name AS area, COUNT(v.id) AS cnt_vac 
FROM 
    vacancies v
LEFT JOIN areas a ON a.id = v.area_id
GROUP BY a.name
ORDER BY 2 DESC
'''

In [13]:
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\347208320.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,area,cnt_vac
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query_4_2 = f'''
SELECT 
    COUNT(*) AS vac_with_zar
FROM 
    vacancies
WHERE 
    salary_from IS NOT NULL or salary_to IS NOT NULL
'''

In [15]:
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\2168503709.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,vac_with_zar
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
query_4_3 = f'''
SELECT 
    ROUND(AVG(salary_from)) AS avg_min, ROUND(AVG(salary_to)) AS avg_max
FROM 
    vacancies
WHERE 
    salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [17]:
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\2222372203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,avg_min,avg_max
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
query_4_4 = f'''
SELECT 
    schedule, employment, COUNT(*)
FROM 
    vacancies
GROUP BY 1, 2
ORDER BY 3 DESC
'''

In [19]:
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\2981602351.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query_4_5 = f'''
SELECT 
    experience, COUNT(*)
FROM 
    vacancies
GROUP BY 1
ORDER BY 2 ASC
'''

In [21]:
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\2601166264.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

ВЫВОДЫ: Основная часть вакансий ожидаемо приходится на крупные города, Москва - лидер вакансий. В среднем, вилка зарплат приходится на диапазон 71-110 т.р.. Львиная доля вакансий с требованием полной занятости. Большаинство вакансий приходиться на сегмент с требуемым опытом не менее одного года. 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
query_5_1 = f'''
SELECT 
    e.name AS company, COUNT(*) AS cnt_vac
FROM
    employers e
JOIN vacancies v on v.employer_id = e.id
GROUP BY e.name
ORDER BY 2 desc
LIMIT 5
'''

In [23]:
df = pd.read_sql_query(query_5_1, connection)
print(df.to_string(index=False))

      company  cnt_vac
       Яндекс     1933
   Ростелеком      491
     Тинькофф      444
         СБЕР      428
Газпром нефть      331


C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3517807715.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
query_5_2 = f'''
SELECT 
    a.name,
    count(distinct e.id) as employers,
    count(distinct v.id) as vacancy
FROM areas a
    LEFT JOIN employers e ON a.id = e.area
    LEFT JOIN vacancies v ON v.area_id = a.id
WHERE 
    v.id is null
GROUP BY a.id
order BY employers DESC
LIMIT 5
'''

In [25]:
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3341481329.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,name,employers,vacancy
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
query_5_3 = f'''
SELECT
       e.name AS employer,
       COUNT(DISTINCT v.area_id) AS cnt_areas
FROM vacancies v
LEFT JOIN employers e ON v.employer_id = e.id
GROUP BY e.name
ORDER BY 2 DESC
LIMIT 5
'''

In [27]:
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\2398953733.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,employer,cnt_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
query_5_4 = f'''
SELECT 
    COUNT(e.id) AS employers_without_industry
FROM 
    employers e 
LEFT JOIN employers_industries i ON e.id = i.employer_id
WHERE i.employer_id IS NULL
'''

In [29]:
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3114411492.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,employers_without_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
query_5_5 = f'''
SELECT 
    e.name AS company,
    COUNT(i.employer_id) AS cnt
FROM 
    Employers_Industries i 
LEFT JOIN employers e ON e.id = i.employer_id
GROUP BY e.name
HAVING COUNT(i.employer_id) = 4
ORDER BY e.name
OFFSET 2
LIMIT 1
'''

In [31]:
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\1650419958.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,company,cnt
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
query_5_6 = f'''
SELECT 
    i.name AS industry,
    COUNT(*) AS cnt
FROM 
    industries i 
LEFT JOIN employers_industries e ON i.id = e.industry_id
WHERE 
    i.name = 'Разработка программного обеспечения'
GROUP BY i.name
'''

In [33]:
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\1169039133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,industry,cnt
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионники_России'
response = requests.get(url)

page = BeautifulSoup(response.text, 'html.parser')
data = []
HTML_data = page.find_all("table")[0].find_all("tr")[1:]
for element in HTML_data:
    sub_data = []
    for i, k in enumerate(element):
        try:
            if i == 1:
                sub_data.append(k.get_text())
        except:
            continue
    data. append(sub_data)
data_result = []
for elem in data:
    data_result.extend(elem)
data_result = tuple(data_result)
data_result

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [35]:
query_5_7 = f'''
WITH yand(town, cnt) AS 
(SELECT 
    a.name,
    count(v.id)
FROM vacancies v
LEFT JOIN employers e ON v.employer_id = e.id
LEFT JOIN areas a ON a.id = v.area_id
WHERE e.name = 'Яндекс' 
    AND a.name IN {data_result} 
    AND v.id IS NOT NULL
GROUP BY a.name
ORDER BY 2 DESC)

(SELECT town, cnt
FROM yand 
ORDER BY cnt ASC)

UNION ALL

(SELECT 'total', SUM(cnt)
FROM yand)

'''

In [36]:
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3129374555.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,town,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Казань,25.0
5,Ростов-на-Дону,25.0
6,Пермь,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

ВЫВОДЫ: Из анализа работодателей следует, что на рынке труда имеется около 3,5 тыс. работодателей, занимающихся разработкой ПО и находящихся в поиске сотрудников. Также, исходя из анализа видно, что более 8 тыс.работодателей не указали область своей деятельностиб хотя некоторые компании, такие, как 2ГИС указали четыре сферы деятельности. Несомненным лидером как по количеству вакансий (почти 2 тыс.), так и количеству регионов, в которых они размещены, является компания Яндекс. Почти пятьсот объявлений этой компании приходится на 16 городов-миллионников.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
query_6_1 = f'''
SELECT COUNT(*) AS DS_cnt
FROM vacancies
WHERE name ILIKE '%data%' 
OR name ILIKE '%данн%'
'''

In [38]:
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\485020324.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,ds_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
query_6_2 = f'''
SELECT COUNT(*)
FROM vacancies
WHERE ((name ILIKE '%data scientist%')
    OR (name ILIKE '%data science%')
    OR (name ILIKE '%исследователь данных%')
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
    OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%'))
    AND ((name ILIKE '%junior%'
    OR experience = 'Нет опыта'
    OR employment = 'Стажировка'))
'''

In [40]:
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3641650667.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# вариант кода ниже дает неверное значение(229), которое принимает система
# верное значение 201, для его получения нужно часть строки "OR (name ILIKE '%ML% ..." заменить на "OR (name LIKE '%ML%" ..."
query_6_3 = f'''
SELECT COUNT(*) AS cnt_sql
FROM vacancies
WHERE ((name ILIKE '%data scientist%')
    OR (name ILIKE '%data science%')
    OR (name ILIKE '%исследователь данных%')
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
    OR (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%'))
    AND (key_skills ILIKE '%SQL%'
    OR key_skills ILIKE '%postgres%')
'''

In [42]:
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\1159587760.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,cnt_sql
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# та же ситуация с ответом, причина аналогична
query_6_4 = f'''
SELECT COUNT(*) AS cnt_python
FROM vacancies
WHERE ((name ILIKE '%data scientist%')
    OR (name ILIKE '%data science%')
    OR (name ILIKE '%исследователь данных%')
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
    OR (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%'))
    AND (key_skills ILIKE '%Python%')
'''

In [44]:
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\972386973.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,cnt_python
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
query_6_5 = f'''
SELECT
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1),2) AS avg_skills_for_DS
FROM
    vacancies
WHERE 
    (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR (name NOT ILIKE '%HTML%' AND name LIKE '%ML%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%%')
    AND key_skills IS NOT NULL
'''

In [46]:
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\390248827.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,avg_skills_for_ds
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
query_6_6 = f'''
SELECT
    experience, ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to)), 0) AS avg_salary_DS
FROM vacancies
WHERE
(
    (name ILIKE '%data scientist%')
    OR (name ILIKE '%data science%')
    OR (name ILIKE '%исследователь данных%')
    OR (name LIKE '%ML%' AND name NOT iLIKE '%html%')
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
GROUP BY experience
'''

In [48]:
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\3561045109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,avg_salary_ds
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,145236.0
3,От 3 до 6 лет,243115.0


***

ВЫВОДЫ: Мы обнаружили, что из всех резюме в базе данных, 1771 вакансия имеет отношение к DS, из этого количества резюме 51 вакансия подходит для начинающего дата-сайентиста. Также, из имеющих отношение к DS вакансий, для 229(201) вакансий в качестве ключевого навыка указан SQL или postgre и 357(351) вакансий, где в качестве ключевого навыка указан Python. Также, выяснили, что в среднем в вакансиях для DS указывается 6-7 ключевых навыков, а средняя зарплата, на которую может рассчитывать претендент с опытом 3-6 лет находится в районе 250т.р


# Общий вывод по проекту

Из анализа вакансий основными выводами можно считать:
1. Для получения рабочей вакансии DS необходимо обладать в среднеи 6-7 ключевыми навыками, среди которых навыки программирования на Python, умение работать с SQL или postgre.
2. Из анализа следует, что рынок вакансий DS представлен в данной базе 1771 предложениями, из этого количества 51 вакансия подходит для новичка без опыта.
3. Также выяснили, что около 3,5 тыс. работодателей, осуществляющих свою деятельность в сфере ИТ, находятся в поиске сотрудников. 
4. Большая часть вакансий с требуемым опытом не менее одного года и требованием полной занятости.

In [49]:
# В качестве дополнительного исследования по работодателям, решил провести анализ компаний с местными (г. Ярославль) вакансиями.
# Вывод ограничил первыми 10 компаниями с наибольшим количеством вакансий.

query_local_1 = f'''
SELECT 
    e.name AS company, a.name AS from, COUNT(v.id) AS vacancy_cnt
FROM
    employers e
JOIN vacancies v ON v.employer_id = e.id
JOIN areas a ON a.id = e.area
WHERE v.area_id = 112 /* указываем регион 'Ярославль'*/
    AND ((v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%')
    or (v.key_skills ILIKE '%SQL%'
    OR v.key_skills ILIKE '%postgres%'))
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 10
'''

In [50]:
df = pd.read_sql_query(query_local_1, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\384786704.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_local_1, connection)


,company,from,vacancy_cnt
0,Компания Стек,Ярославль,4
1,Diasoft,Москва,4
2,"КРИСТА, НПО",Рыбинск,4
3,РТ Лабс,Москва,3
4,Северсталь,Россия,2
5,Совкомбанк Технологии,Москва,2
6,Тензор,Ярославль,2
7,DIKIDI,Ярославль,2
8,Промсвязьбанк,Москва,2
9,IBS,Москва,1


In [51]:
# Подробнее о местных вакансиях без требований к опыту работы.

query_local_2 = f'''
SELECT 
    e.name AS company, a.name AS from, v.name, REPLACE(key_skills, CHR(9), ', '), v.schedule, v.experience, v.employment, v.salary_from, v.salary_to
FROM
    employers e
JOIN vacancies v on v.employer_id = e.id
JOIN areas a on a.id = e.area
WHERE v.area_id = 112 /* указываем регион 'Ярославль'*/
    AND v.experience = 'Нет опыта'
    AND ((v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%')
    or (v.key_skills ILIKE '%SQL%'
    OR v.key_skills ILIKE '%postgres%'))
'''

In [52]:
df = pd.read_sql_query(query_local_2, connection)
df

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_16120\437803806.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_local_2, connection)


,company,from,name,replace,schedule,experience,employment,salary_from,salary_to
0,"КРИСТА, НПО",Рыбинск,Java-разработчик,"Java, ORACLE, Redmine, SQL, ООП",Полный день,Нет опыта,Полная занятость,NaN,NaN
1,"КРИСТА, НПО",Рыбинск,Javascript-разработчик,"JavaScript, HTML, SQL, CSS",Полный день,Нет опыта,Полная занятость,NaN,NaN
2,"КРИСТА, НПО",Рыбинск,Специалист по сопровождению программного обесп...,"Работа с базами данных, Сопровождение клиентов...",Полный день,Нет опыта,Полная занятость,NaN,NaN
3,Компания Стек,Ярославль,Программист-стажер,"SQL, MS SQL, MS SQL Server, JavaScript",Полный день,Нет опыта,Частичная занятость,30000.0,70000.0
4,Diasoft,Москва,Специалист по внедрению (ПО для финансовых рын...,"SQL, Английский язык, Scrum, MS SQL, MS SQL Se...",Полный день,Нет опыта,Полная занятость,NaN,NaN
5,Лига Цифровой Экономики,Москва,DWH\ETL-администратор,"СУБД, SQL, Linux, Базы данных",Полный день,Нет опыта,Полная занятость,NaN,NaN
6,РТ Лабс,Москва,Разработчик услуг (junior),"SQL, Git, XML, HTML, ООП, JSON",Полный день,Нет опыта,Полная занятость,NaN,NaN
7,Компания Стек,Ярославль,PHP-программист (Web-разработчик),"PHP, HTML, CSS, MySQL, JavaScript, Bootstrap, ...",Полный день,Нет опыта,Полная занятость,60000.0,NaN
8,Avanta Россия,Москва,Программист 1С,"1С: Предприятие 8, SQL, 1С программирование, 1...",Полный день,Нет опыта,Полная занятость,NaN,NaN


ВЫВОДЫ: Дополнительный анализ показал достаточное количество вакансий от местных работодателей, имеющих отношение к анализу данных или работе с СУБД, хотя и не все эти вакансии напрямую имеют отношение к DS. Было интересно получить общую картину по местным вакансиям данного направления.